# ENGRI 1120 Lab 8: Analysis of continuous stirred tank reactor with a single irreversible chemical reaction

<img src="figs/Fig-Lab-8-Reactor.png" style="width:40%">

##### __Fig. 1__. Chemical reactor schematic. A single liquid phase reaction produces the product $P$ from the precursor molecules $A$ and $B$.

### Introduction
Suppose we have a reaction set $\mathcal{R}$ involving the species set $\mathcal{M}$ that is occurring in a well-mixed chemical reactor with stream set $\mathcal{S}$. Then the concentration of component $i\in\mathcal{M}$ is given by:

$$\frac{d}{dt}\left(C_{i}V\right) = \sum_{s\in\mathcal{S}}v_{s}C_{s,i}\dot{V}_{s} + \sum_{r\in\mathcal{R}}\sigma_{ir}\hat{r}_{r}V\qquad\forall{i}\in\mathcal{M}$$

where $C_{s,i}$ denotes the concentration of component $i\in\mathcal{M}$ in stream $s\in\mathcal{S}$, $\sigma_{ir}$ denotes the stoichiometric coefficient for component $i\in\mathcal{M}$ in reaction $r\in\mathcal{R}$, $\dot{V}_{s}$ denotes the volumetric flow rate of stream $s\in\mathcal{S}$, $V$ denotes the volume of the reaction mixture in the reactor unit, $v_{s}$ denotes the direction parameter for stream $s\in\mathcal{S}$ and the quantity $C_{i}$ denotes the concentration of component $i\in\mathcal{M}$ in the reaction vessel. Finally, the terms $\hat{r}_{r}$ denote the _reaction rate per unit volume_ for reaction $r\in\mathcal{R}$ that is occurring in the reaction vessel. 

#### Constant volume, steady-state:
When the reaction is at constant volume, we can pull the volume $V$ out of the accumulation term and divide by the volume to give:

$$\frac{dC_{i}}{dt} = \sum_{s\in\mathcal{S}}v_{s}C_{s,i}D_{s} + \sum_{r\in\mathcal{R}}\sigma_{ir}\hat{r}_{r}\qquad\forall{i}\in\mathcal{M}$$

where $D_{s}$ is called the _dilution rate_ for stream $s\in\mathcal{S}$; the dilution rate has units of inverse time. Finally, at steady-state, all accumulation terms vanish, giving:

$$\sum_{s\in\mathcal{S}}v_{s}C_{s,i}D_{s} + \sum_{r\in\mathcal{R}}\sigma_{ir}\hat{r}_{r} = 0\qquad\forall{i}\in\mathcal{M}$$

#### Problem setup and assumptions
Let's do some calculations for the reactor shown in Fig. 1; an irreversible liquid phase reaction occurs, which converts $A$ and $B$ into product $P$ according to the reaction:

$$2A+B\longrightarrow{P}$$

at the rate $\hat{r}_{1}$ (units: mol/volume-time). The volume of the reaction mixture $V$ = 5L. Stream 1 has a volumetric flow rate $\dot{V}_{1}$ = 200 mL/h and Stream 2 has a volumetric flow rate $\dot{V}_{2}$ = 300 mL/h. The concentration(s) of $A$ in the stream 1 is $C_{1,1}$ = 50 mmol/L, there is no $B$ and $C$ in stream $1$. The concentration(s) of $B$ in the stream 2 is $C_{2,2}$ = 75 mmol/L, there is no $A$ or $C$ in stream $2$. The rate constant for rate $1$ is given by: $k_{1}$ = 0.56 units. 

__Assumptions__:
* The reactor is well-mixed
* The rate of reaction for $\hat{r}_{1}$ follows mass action kinetics
* Let A = index 1, B = index 2 and C = index 3

### Lab setup
The code block below installs (and loads) [Julia](https://julialang.org) packages that we use to solve the species concentration balance equations.

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `C:\Users\ortiz\Documents\GitHub\ENGRI-1120-IntroToChemE-Example-Notebooks\labs\lab-8-fun-w-mass-action-kinetics`
   Installed OffsetArrays ───────────────────── v1.12.7
   Installed SIMDDualNumbers ────────────────── v0.1.1
   Installed TreeViews ──────────────────────── v0.3.0
   Installed Sundials_jll ───────────────────── v5.2.1+0
   Installed NonlinearSolve ─────────────────── v0.3.22
   Installed Polyester ──────────────────────── v0.6.15
   Installed DifferentialEquations ──────────── v7.5.0
   Installed StaticArrays ───────────────────── v1.5.8
   Installed FunctionWrappers ───────────────── v1.1.2
   Installed RecursiveArrayTools ────────────── v2.32.0
   Installed TriangularSolve ────────────────── v0.1.14
   Installed Static ─────────────────────────── v0.7.7
   Installed ZygoteRules ────────────────────── v0.2.2
   Installed ArrayInterfaceOffsetArrays ─────── v0.1.6
   Installed CEnum ──────────────────────────── v0.4.2
   Installed RandomNumbers ───

In [13]:
# load required packages 
using DifferentialEquations
using Optim
using Plots
using Colors

# setup paths -
const _ROOT = pwd();
const _PATH_TO_FIGS = joinpath(_ROOT, "figs");

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1662
  ✓ OrdinaryDiffEq


#### Load the Lab 8 code library
The call to the `include` function loads the `ENGRI-1120-Lab-6-CodeLib.jl` library into the notebook; the library contains functions we can use during the lab. In particular:

#### Functions
* The `kinetics(x::Array{Float64,1}, k::Float64) -> Float64` function computes the value of reaction rate $\hat{r}_{1}$ given the concentrations in the `x::Array{Float64,1}` array and the rate constant `k::Float64`.
* The `evaluate(model::Dict{String, Any}; tspan::Tuple{Float64, Float64} = (0.0,20.0), Δt::Float64 = 0.01) --> (T,X)` computes the numerical solution of the concentration balances; the time is stored in the `T::Array{Float64,1}` array. The concentrations are stored in the `X::Array{Float64,2}` array; each row of `X::Array{Float64,2}` corresponds to a time point, while each column holds the concentration of a species
* The `objfunc(x::Array{Float64,1}, model::Dict{String, Any}) --> Float64` function is called by `Optim` to estimate the `residual` of the concentration balances.

In [ ]:
include("ENGRI-1120-Lab-8-CodeLib-Soln.jl");

InterruptException: InterruptException:

In [ ]:
# setup some constants from the problem -
V = 5.0; # units: L
V̇₁ = 200.0*(1/1000); # units: L/hr
V̇₂ = 300.0*(1/1000); # units: L/hr

# rate constant -
k₁ = 0.56;

# Feed concentrations -
A_in = 50.0; # units: mmol/L
B_in = 75.0; # units: mmol/L
C_in = 0.0;  # units: mmol/L (no product in any feed)

# initial condition in the reactor -
xₒ = [0.1, 0.1, 1e-6];

### Setup task: Write the concentration balances for $A$, $B$ and $C$.

In [ ]:
# use a Markdown cell and write the stready-state concentration balance equations in LaTeX!

# new to LaTex?: look at the introduction, and/or check out
# link: https://typeset.io/resources/learn-latex-beginners-step-by-step-guide/#what-is-latex

#### Specific Concentration Balance

$$\frac{dC_{i}}{dt} = v_{1}C_{1,1}D_{1} + v_{2}C_{2,2}D_{2} - v_{3}C_{3,3}D_{3} + \sum_{r\in\mathcal{R}}\sigma_{ir}\hat{r}_{r}\qquad\forall{i}\in\mathcal{M}$$

### Setup task: Formulate the stoichiometric array, the composition array and the dilution vector

In [29]:
# setup stoichiometric array -
# fill me in ....
    #  A     B    C
S = [-2.0, -1.0, 1.0][:,:]

# setup the dilution rate vector -
# fill me in ...
D = [


# setup concentration array -
# the concentration array will be a species x streams array holding concentration values (put all zeros for the exit streams)
# fill me in ...

LoadError: syntax: incomplete: premature end of input

### Setup task: Put all our data into a `model` dictionary
Dictionary is a fantastic data structure [check out the documentation for more information](https://docs.julialang.org/en/v1/base/collections/#Dictionaries). 

In [ ]:
# initialize Dict data structrure -
model = Dict{String, Any}();

# data set by you in the model dictionary
# fill me in ... check the CodeLib to see what the evaluate method is expecting ...

### a) Estimate of the dynamic concentration for a constant feed
To estimate the `dynamics` in the reactor, we need to numerically solve the differential concentration balance equations, e.g., using methods such as the [Euler method](https://en.wikipedia.org/wiki/Euler_method). We'll use more sophisticated methods found in [DifferentialEquations.jl](https://diffeq.sciml.ai/stable/). To interface with these solvers, use the `evaluate` function imported from `ENGRI-1120-Lab-8-CodeLib-Soln.jl`.

Solve the system for 30 hours of time, with a `saveat` value of 0.01

In [ ]:
# Call the evalaute method here -
# fill me in ...

# make a plot of time versus the concentration of A, B and C

### b) Estimate the steady state exit composition as an optimization problem
To estimate the steady-state concentration, we need to solve an `optimization` problem, i.e., we need to `search` for exit concentrations that make our concentration balances zero. We do this via the [Optim.jl](https://julianlsolvers.github.io/Optim.jl/stable/) package. The problem we are solving is to find a concentration vector $x$ that makes the residual $\epsilon$ small:

$$\min_{x}\epsilon^{T}\epsilon$$

subject to the constraints on the concentration $0\leq{x}\leq\infty$. We'll use a derivative-free search method called [Nelder-Mead](https://en.wikipedia.org/wiki/Nelder–Mead_method) to generate candidate values for the concentration vector $x$; we'll keep generating guesses and checking their residual values until we find a candidate solution that meets some smallness criteria.

In [ ]:
# Setup and solve the optimization problem. Hint: look at the HFCS example
# Fill me in.

In [ ]:
# get the optimal solution from the opt_result object.
# fill me in

### c) Visualize the dynamic and steady-state solutions

In [ ]:
# On the same axis, plot the steady-state and dynamic solutions for A, B and C
# fill me in

### d) Wow! This takes a long time to reach steady-state. How can we change this to shorten the time we have to wait?

In [ ]:
# your ideas go here ... check them by running some simulations!